<a href="https://colab.research.google.com/github/Anvarka/Crime_prediction/blob/main/prophet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
! pip install prophet
from prophet import Prophet

  Using cached prophet-1.0.1.tar.gz (65 kB)
  Using cached cmdstanpy-0.9.68-py3-none-any.whl (49 kB)
  Using cached ujson-5.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43 kB)
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6640070 sha256=7a59beb1284182876e070fc4143470154e8eddcabd10c50b5f6baa67c4e201ae
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


 # Данные

In [2]:
#importing dataset
data=pd.read_csv('/content/drive/MyDrive/stl-crime-data_2008-2015.tsv', delimiter='\t', encoding='utf-8')
data = data[['DateOccured', 'Neighborhood', 'UCRCrime']]
data['DateOccured'] = [datetime.strptime(data.iloc[i]['DateOccured'], '%m/%d/%Y %H:%M').date() for i in range(len(data.index))]
data

,DateOccured,Neighborhood,UCRCrime
0,2000-01-01,53,Sex Offenses
1,2003-01-01,17,Larceny-theft
2,2004-01-01,68,Fraud
3,2006-01-01,16,Fraud
4,2007-01-01,57,Other assaults
...,...,...,...
477636,2013-12-01,5,Larceny-theft
477637,2013-12-20,65,Forcible Rape
477638,2014-12-22,14,Other assaults
477639,2014-12-26,60,Robbery


### кодирование типа преступления и места в число:

In [3]:
mp = {}
reverse_mp = {}
i = 0
for type_ in np.unique(data["UCRCrime"]):
    for place_ in np.unique(data["Neighborhood"]):
        mp[(type_, place_)] = i
        reverse_mp[i] = (type_, place_)
        i += 1

In [4]:
from tqdm import tqdm
type_place = data[["UCRCrime", "Neighborhood"]]
l = []
for index, row in tqdm(type_place.iterrows()):
    l.append(mp[(row["UCRCrime"], row['Neighborhood'])])

449000it [00:39, 11263.14it/s]


### Создание датасета для prophet-a, в котором дата преступления и закодированные тип преступления и место. 

In [5]:
data_prophet = pd.DataFrame()
data_prophet["ds"] = data["DateOccured"]
data_prophet["y"] = l
data_prophet

,ds,y
0,2000-01-01,1922
1,2003-01-01,1174
2,2004-01-01,1047
3,2006-01-01,995
4,2007-01-01,1659
...,...,...
477636,2013-12-01,1162
477637,2013-12-20,866
477638,2014-12-22,1616
477639,2014-12-26,1840


### Берем данные с 2008 года

In [6]:
#keeping dates starting 2008
start = datetime.strptime('Jan 1 2008 00:00','%b %d %Y %H:%M').date()
filt = data_prophet['ds'] >= start
data_prophet = data_prophet.where(filt)
data_prophet = data_prophet.dropna()
data_prophet.isna().sum()

ds    0
y     0
dtype: int64

### Делим данные на train/test 

In [7]:
test_start = datetime.strptime('Jan 1 2015 00:00','%b %d %Y %H:%M').date()
filt_test = data_prophet['ds'] >= test_start
test_data_prophet = data_prophet.where(filt_test)
test_data_prophet = test_data_prophet.dropna()
test_data_prophet.isna().sum()

train_start = datetime.strptime('Jan 1 2015 00:00','%b %d %Y %H:%M').date()
filt_train = data_prophet['ds'] < train_start
data_prophet = data_prophet.where(filt_train)
data_prophet = data_prophet.dropna()
data_prophet.isna().sum()

ds    0
y     0
dtype: int64

# Обучение модели

In [8]:
model = Prophet()
model.fit(data_prophet)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


# Предсказание модели

In [9]:
future = model.make_future_dataframe(periods = 365)
forecast = model.predict(future)

### Декодирование предсказанного числа в тип и место преступления

In [ ]:
t = forecast[["ds", "yhat"]]
types = []
places = []
for index, row in t.iterrows():
    type_, place_ = reverse_mp[int(row["yhat"])]
    types.append(type_)
    places.append(place_)

t["crime"] = types
t["neighbord"] = places
t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,yhat,crime,neighbord
0,2008-01-01,1194.445004,Larceny-theft,37
1,2008-01-02,1197.958348,Larceny-theft,40
2,2008-01-03,1199.281174,Larceny-theft,42
3,2008-01-04,1208.987987,Larceny-theft,51
4,2008-01-05,1226.555089,Larceny-theft,69
...,...,...,...,...
2917,2015-12-27,1223.090838,Larceny-theft,66
2918,2015-12-28,1199.488905,Larceny-theft,42
2919,2015-12-29,1195.733358,Larceny-theft,38
2920,2015-12-30,1199.706897,Larceny-theft,42


# Метрика

In [ ]:
# date : (set(тип), set(место))
date_to_type_place = {}
for index, row in data.iterrows():
    types_, places_ = date_to_type_place.get(str(row["DateOccured"]), (set(), set()))
    types_.add( row["UCRCrime"])
    places_.add(row["Neighborhood"])
    date_to_type_place[str(row["DateOccured"])] = (types_, places_)

In [ ]:
date_to_type_place

In [ ]:
print(t['ds'][0])
print(test_start)

2008-01-01 00:00:00
2015-01-01


In [ ]:
test_start = datetime.strptime('Jan 1 2015 00:00','%b %d %Y %H:%M').date()
t['ds'] = t['ds'].apply(lambda x: x.date())
filt_pred = t['ds'] >= test_start
pred = t.where(filt_pred)
pred = pred.dropna()
pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,yhat,crime,neighbord
2557,2015-01-01,1208.509257,Larceny-theft,51.0
2558,2015-01-02,1218.638552,Larceny-theft,61.0
2559,2015-01-03,1236.660583,Larceny-theft,79.0
2560,2015-01-04,1239.208419,Larceny-theft,82.0
2561,2015-01-05,1214.484545,Larceny-theft,57.0
...,...,...,...,...
2917,2015-12-27,1223.090838,Larceny-theft,66.0
2918,2015-12-28,1199.488905,Larceny-theft,42.0
2919,2015-12-29,1195.733358,Larceny-theft,38.0
2920,2015-12-30,1199.706897,Larceny-theft,42.0


In [ ]:
metrics = []
for index, row in pred.iterrows():
    types_, places_ = date_to_type_place.get(str(row["ds"]), (None, None))
    score = 0
    if types_ is None:
        continue
    if row["crime"] in types_:
        score += 0.5
    if row["neighbord"] in places_:
        score += 0.5
    metrics.append(score)

In [ ]:
print("len(metrics)=", len(metrics))
print("sum(metrics)=", sum(metrics))
print("mean(metrics)=", np.mean(metrics))

len(metrics)= 365
sum(metrics)= 292.5
mean(metrics)= 0.8013698630136986
